In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from keras.datasets import mnist


print(f"Tensorflow {tf.__version__}")

2025-12-03 11:12:01.378862: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-03 11:12:01.386986: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-03 11:12:01.934616: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-03 11:12:05.443872: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

Tensorflow 2.20.0


In [2]:

def cnn_with_three_blocks():

    inputs = tf.keras.Input(shape=(28, 28, 1))

    # Block 1
    x = Conv2D(32, (3, 3), padding='same', activation='relu', use_bias=False)(inputs)
    x = Conv2D(32, (3, 3), padding='same', activation='relu', use_bias=False)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Block 2
    x = Conv2D(64, (3, 3), padding='same', activation='relu', use_bias=False)(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu', use_bias=False)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    # Block 3
    x = Conv2D(128, (3, 3), padding='same', activation='relu', use_bias=False)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.4)(x)

    # Classifier
    x = Flatten()(x)
    x = Dense(256, use_bias=False)(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(10, activation='softmax', use_bias=False)(x)


    # Create functional model
    model= Model(inputs, outputs)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

    # Loss function
    model.compile(
        loss='categorical_crossentropy', # loss function
        optimizer=optimizer, # learning rule
        metrics=['accuracy'] # show accuracy
    )
    print(model.summary())

    return model

In [3]:
# Download data
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()

# Normalize inputs
train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

# One-hot output vectors
train_labels_onehot = tf.keras.utils.to_categorical(train_labels, 10)
test_labels_onehot = tf.keras.utils.to_categorical(test_labels, 10)

In [4]:
model = cnn_with_three_blocks()

# Train model (use one-hot labels)
history = model.fit(
    train_data, train_labels_onehot,       # training data (one-hot labels)
    batch_size=128,                        # batch size
    epochs=20,                              # Maximum number of epochs
    validation_split=0.1,                  # Percentage of training data used for validation
)

# Test model: get class predictions and evaluate with one-hot labels
predictions_keras = np.argmax(model.predict(test_data, verbose=0), axis=1)
test_loss, test_accuracy = model.evaluate(test_data, test_labels_onehot, verbose=0)
print(f"Test accuracy: {test_accuracy}")
model.save("MNIST_v4.keras")

2025-12-03 11:12:09.950187: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 32)     │         9,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 14, 14, 64)     │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 14, 14, 64)     │        36,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 7, 7, 128)      │        73,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       294,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,560 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 436,000 (1.66 MB)

 Trainable params: 436,000 (1.66 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 53s 123ms/step - accuracy: 0.1252 - loss: 2.2906 - val_accuracy: 0.1877 - val_loss: 2.2357
Epoch 2/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 50s 119ms/step - accuracy: 0.3811 - loss: 1.7639 - val_accuracy: 0.8797 - val_loss: 0.5703
Epoch 3/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 52s 122ms/step - accuracy: 0.7686 - loss: 0.6923 - val_accuracy: 0.9503 - val_loss: 0.2078
Epoch 4/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 50s 119ms/step - accuracy: 0.8652 - loss: 0.4249 - val_accuracy: 0.9625 - val_loss: 0.1384
Epoch 5/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 50s 119ms/step - accuracy: 0.8971 - loss: 0.3286 - val_accuracy: 0.9673 - val_loss: 0.1134
Epoch 6/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 53s 125ms/step - accuracy: 0.9149 - loss: 0.2733 - val_accuracy: 0.9683 - val_loss: 0.1058
Epoch 7/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 51s 120ms/step - accuracy: 0.9239 - loss: 0.2452 - val_accuracy: 0.9738 - val_loss: 0.0904
Epoch 8/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 54s 127ms/step - accuracy: 0.9296 - lo

In [5]:
import ANNarchy
from ANNarchy.extensions.ann_to_snn_conversion import ANNtoSNNConverter
ANNarchy.clear()
snn_converter = ANNtoSNNConverter(
    input_encoding='IB', 
    hidden_neuron='IaF',
    read_out='spike_count',
)


ANNarchy 4.8 (4.8.2.5) on linux (posix).


In [6]:
net = snn_converter.load_keras_model("MNIST_v4.keras", show_info=True)

* Input layer: input_layer, (28, 28, 1)
* InputLayer skipped.
* Conv2D layer: conv2d, (28, 28, 32) 
* Conv2D layer: conv2d_1, (28, 28, 32) 
* MaxPooling2D layer: max_pooling2d, (14, 14, 32) 
* Dropout skipped.
* Conv2D layer: conv2d_2, (14, 14, 64) 
* Conv2D layer: conv2d_3, (14, 14, 64) 
* MaxPooling2D layer: max_pooling2d_1, (7, 7, 64) 
* Dropout skipped.
* Conv2D layer: conv2d_4, (7, 7, 128) 
* MaxPooling2D layer: max_pooling2d_2, (3, 3, 128) 
* Dropout skipped.
* Flatten skipped.
* Dense layer: dense, 256 
    weights: (256, 1152)
    mean -0.00047615289804525673, std 0.037848860025405884
    min -0.1071130633354187, max 0.10457154363393784
* Activation skipped.
* Dropout skipped.
* Dense layer: dense_1, 10 
    weights: (10, 256)
    mean -0.00029122334672138095, std 0.09287582337856293
    min -0.23552492260932922, max 0.23119144141674042



In [7]:
predictions_snn = snn_converter.predict(test_data[:300], duration_per_sample=200)

100%|██████████| 300/300 [24:10<00:00,  4.83s/it]


In [8]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

print(classification_report(test_labels[:300], predictions_snn))
print("Test accuracy of the SNN:", accuracy_score(test_labels[:300], predictions_snn))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        24
           1       1.00      1.00      1.00        41
           2       1.00      1.00      1.00        32
           3       1.00      1.00      1.00        24
           4       1.00      0.97      0.99        37
           5       1.00      1.00      1.00        29
           6       0.96      0.96      0.96        24
           7       1.00      1.00      1.00        34
           8       1.00      1.00      1.00        21
           9       1.00      1.00      1.00        34

    accuracy                           0.99       300
   macro avg       0.99      0.99      0.99       300
weighted avg       0.99      0.99      0.99       300

Test accuracy of the SNN: 0.9933333333333333


In [9]:
model = tf.keras.models.load_model('MNIST_v4.keras')
loss, accuracy = model.evaluate(test_data, test_labels_onehot, verbose=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9845 - loss: 0.0459


In [10]:
print(f"Test accuracy of the ANN: {accuracy:.4f}")
print("Test accuracy of the SNN:", accuracy_score(test_labels[:300], predictions_snn))

Test accuracy of the ANN: 0.9845
Test accuracy of the SNN: 0.9933333333333333
